# **Homogeneous parking prices**

In [ ]:
import json

In [ ]:
with open("../02_data/starting_prices.json") as f:
    starting_prices = json.load(f)

In [ ]:
for p in starting_prices:
    starting_prices[p] = 200

In [ ]:
with open("../02_data/starting_prices.json", "w") as f:
    json.dump(starting_prices, f)

## Results

In [ ]:
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
MEAS_ROOT = "../02_data/"
SEEDS = [42, 1812, 9698, 424, 820, 75, 98, 65535, 16383, 513]
PENS = [1, 2, 3, 4, 5, 6, 7, 8, 9]
EPSILON = 10

In [ ]:
EUR_HUF = 411.43 #2024-11-24

In [ ]:
def read_data(seed, meas_path, col_name, col_value):
    with open(f"{meas_path}/veh_results.json") as f:
        veh_res = json.load(f)
    veh_data = pd.DataFrame.from_dict(veh_res, orient="index")
    veh_data.index = veh_data.index.astype(int)
    
    trip_df = pd.read_xml(f"{meas_path}/vehicle_trips.xml", xpath = "tripinfo")
    trip_df["id"] = trip_df["id"].astype(int)
    
    vd = pd.merge(left=trip_df, right=veh_data, left_on="id", right_index=True)
    
    vd[col_name] = [col_value]*len(vd)
    
    return vd

In [ ]:
#read penetration data:

penetration_df = pd.DataFrame()

for seed in SEEDS:
    penetration_df = pd.concat([penetration_df,
                            read_data(seed, f"{MEAS_ROOT}/ideal{seed}/", "penetration", "100%")
                           ], ignore_index=True)
    
for seed in SEEDS:
    penetration_df = pd.concat([penetration_df,
                            read_data(seed, f"{MEAS_ROOT}/baseline{seed}/", "penetration", "0%")
                           ], ignore_index=True)
    
for seed in SEEDS:
    info_df = read_data(seed, f"{MEAS_ROOT}/information{seed}/", "penetration", "info.")
    info_df["controlled"] = False
    info_df["paid_price"] = info_df["parking_price"]
    penetration_df = pd.concat([penetration_df, info_df], ignore_index=True)
    
for pen in PENS:
    for seed in SEEDS:
        penetration_df = pd.concat([penetration_df,
                                read_data(seed, f"{MEAS_ROOT}/pen{pen}:{seed}/", "penetration", f"{pen*10}%")
                               ], ignore_index=True)

In [ ]:
penetration_df["prolonged_distance"] = penetration_df["routeLength"] - penetration_df["original_distance"]

penetration_df["paid_price"] = penetration_df["paid_price"]/EUR_HUF
penetration_df["auction_price"] = penetration_df["auction_price"]/EUR_HUF

In [ ]:
labels = ["0%", "info."] + [f"{i}%" for i in range(10, 110, 10)]

In [ ]:
values = []
value_types = []
pens = []
controlled = []

for _,r in penetration_df.iterrows():
    values.append(r["rerouteNo"])
    value_types.append("n_reroutes")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["routeLength"])
    value_types.append("distance")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["prolonged_distance"])
    value_types.append("prolonged_distance")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["parking_distance"])
    value_types.append("parking_distance")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["paid_price"])
    value_types.append("paid_price")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["auction_price"])
    value_types.append("auction_price")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
    values.append(r["occupied_reserved"])
    value_types.append("occupied_reserved")
    controlled.append(r["controlled"])
    pens.append(r["penetration"])
    
long_df = pd.DataFrame()
long_df["values"] = values
long_df["value_types"] = value_types
long_df["controlled"] = controlled
long_df["penetration"] = pens

In [ ]:
total_parking_prices = []
participating_parking_prices = []
not_participating_parking_prices = []

for pen in labels:
    df_filt = long_df[long_df["penetration"] == pen]
    pp = df_filt[df_filt["value_types"] == "paid_price"]
    total_parking_prices.append(pp["values"].sum())
    participating_parking_prices.append(pp[pp["controlled"] == True]["values"].sum())
    not_participating_parking_prices.append(pp[pp["controlled"] == False]["values"].sum())

In [ ]:
total_parking_prices = np.array(total_parking_prices)/10
participating_parking_prices = np.array(participating_parking_prices)/10
not_participating_parking_prices = np.array(not_participating_parking_prices)/10

participating_parking_prices[participating_parking_prices == 0] = None
not_participating_parking_prices[not_participating_parking_prices == 0] = None

In [ ]:
plt.figure(figsize=(12,4))

plt.hlines(y=max(total_parking_prices), xmin=0, xmax=len(labels)-1, linestyle="--", color="k", alpha=.33)
plt.hlines(y=min(total_parking_prices), xmin=0, xmax=len(labels)-1, linestyle="--", color="k", alpha=.33,
           label="revenue limits")

plt.plot(labels, total_parking_prices, label="all vehicles", linewidth=2)
plt.scatter(labels, total_parking_prices)
plt.plot(labels, participating_parking_prices, label="participating vehicles", linewidth=2)
plt.scatter(labels, participating_parking_prices)
plt.plot(labels, not_participating_parking_prices, label="not particpating vehicles", linewidth=2)
plt.scatter(labels, not_participating_parking_prices)

plt.xlabel("penetration")
plt.ylabel("total revenue [€]")
plt.legend()

In [ ]:
plt.plot(labels, total_parking_prices)

In [ ]:
total_parking_prices_reflective = [6901.27117615, 6888.19483266, 6817.98118757, 6692.97328829,
       6534.12731206, 6332.76863622, 6118.2922976 , 5918.1683397 ,
       5726.6339353 , 5573.07926014, 5501.24687067, 5534.70335173]

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(labels, total_parking_prices, label="homogeneous", linewidth=2)
plt.scatter(labels, total_parking_prices)
plt.plot(labels, total_parking_prices_reflective, label="demand-driven", linewidth=2)
plt.scatter(labels, total_parking_prices_reflective)
plt.legend(title="starting prices")
plt.xlabel("penetration")
plt.ylabel("total revenue [€]")